### 2000 U.S. Presidential Election in Florida

The 2000 presidential election was contested by George W. Bush (R) and Al Gore (D). It was an extraordinarily close race and came down to just one state: Florida. Because of the Electoral College, whoever won Florida would win the election.

In this Colab, you will explore the Florida election data by county to understand the controversies surrounding this election.

In [57]:
import pandas as pd
#https://colab.research.google.com/drive/1z0hCqfi9bOqmcRr5t-tEL1SraQTI0HVU?usp=sharing#scrollTo=vGWFfwY84IKD
# This overrides how Colab displays Pandas Series.
#from google.colab import _reprs
#_reprs.disable_pandas_series_repr

# Question 1
The data can be found at https://datasci112.stanford.edu/data/florida2000.csv. Read in the data. The columns Bush00, Gore00, Buchanan00, and Other00 contain the number of votes obtained by Bush, Gore, Pat Buchanan (a third-party candidate), and all other candidates, respectively, in the 2000 election.

Who won the election and by how many votes? Notice that some of the votes came from overseas absentee voters. If we exclude these votes, who would have won the election?

In [104]:
# YOUR CODE HERE

df = pd.read_csv("https://datasci112.stanford.edu/data/florida2000.csv")

winner = df[["Bush00", "Gore00", "Buchanan00", "Other00"]].sum(axis="rows").sort_values(ascending=False).first_valid_index()

absentee_excluded_df = df[df["County"] != "(Federal Absentee)"]
absentee_excluded_winner = absentee_excluded_df[["Bush00", "Gore00", "Buchanan00", "Other00"]].sum(axis="rows").sort_values(ascending=False).first_valid_index()

print(f"Winner including overseas absentee voters: {winner}, votes: {df[winner].sum()}")
print(f"Winner excluding overseas absentee voters: {absentee_excluded_winner}, votes: {absentee_excluded_df[absentee_excluded_winner].sum()}")


Winner including overseas absentee voters: Bush00, votes: 2912790
Winner excluding overseas absentee voters: Gore00, votes: 2911417


YOUR INTERPRETATION HERE

When we include oversea voters we can see the the winner Bush but when we exclude oversea voters with `["county"] != (Aederal Absentee)` the winner is Gore

## Question 2
One controversy in the 2000 election was the so-called "butterfly ballot" used by Palm Beach County, shown below.
Gore was the 2nd ticket listed on the left-hand side, but to vote for Gore, a voter had to punch the 3rd hole. Punching the 2nd ticket would result in a vote for Pat Buchanan.

Many voters claimed that they were confused by the ballot design and inadvertently voted for Buchanan, when they meant to vote for Gore.

Learn: Take a look at the plotly scatter() documentation ( https://plotly.com/python-api-reference/generated/plotly.express.scatter.html). There are a ton of optional arguments! Pick 2 arguments besides DataFrame, x, and y and explain what they do.

Do: Make scatterplots of the data to see if there is evidence that there were an unusual number of voters for Pat Buchanan in Palm Beach County. Would this have been enough to swing the election?

In [59]:
# YOUR CODE HERE
import plotly.express as px

px.scatter(df,
            x="Gore00",
            y="Buchanan00",
            title="Gore vs Buchanan votes in Florida precincts",
            color="County"
            ).show()

**Explanation:** 

**Two plotly.express.scatter() arguments:**

1. **`title`**: Sets the title text that appears at the top of the chart. This helps viewers quickly understand what the visualization represents.

2. **`color`**: Assigns different colors to data points based on the values in the specified column. In this case, coloring by "County" creates a unique color for each county, making it easy to identify and distinguish Palm Beach County from the others. It also automatically generates a legend on the right side.

**Analysis:**

Looking at the scatter plot, Palm Beach County is a dramatic outlier. With approximately 269,732 Gore votes, Palm Beach had 3,411 Buchanan votes - far more than any other county with a similar number of Gore voters. Most counties with similar Gore vote counts had only a few hundred Buchanan votes at most.

To determine if this would have swung the election, we can check the margin:

In [60]:
# Calculate the vote margin and Palm Beach Buchanan votes
bush_total = df["Bush00"].sum()
gore_total = df["Gore00"].sum()
margin = bush_total - gore_total

palm_beach_buchanan = df[df["County"] == "Palm Beach"]["Buchanan00"].values[0]

print(f"Bush total votes: {bush_total:,}")
print(f"Gore total votes: {gore_total:,}")
print(f"Bush's margin of victory: {margin:,} votes")
print(f"\nPalm Beach Buchanan votes: {palm_beach_buchanan:,}")
print(f"\nIf even half of Palm Beach's Buchanan voters intended to vote for Gore:")
print(f"  Gore would have gained: {palm_beach_buchanan // 2:,} votes")
print(f"  New margin: {margin - palm_beach_buchanan // 2:,} votes")
print(f"  Winner would have been: {'Gore' if gore_total + palm_beach_buchanan // 2 > bush_total else 'Bush'}")

Bush total votes: 2,912,790
Gore total votes: 2,912,253
Bush's margin of victory: 537 votes

Palm Beach Buchanan votes: 3,411

If even half of Palm Beach's Buchanan voters intended to vote for Gore:
  Gore would have gained: 1,705 votes
  New margin: -1,168 votes
  Winner would have been: Gore


## Question 3a
Another controversy in the 2000 election was overvotes. Overvoting is when a voter selects more than one candidate. Overvotes are disqualified and not counted. In the 2000 election, Gore appeared on a disproportionate number of overvotes in Florida.

Calculate the overvote proportion in each county.


**IMPORTANT NOTE** In this question I couldn't understand the demanded proportion. Is it overvote proportion to all voters in a county or the overvote proportion to total overvotes for each county?. So I calculated both of them.

In [130]:
# YOUR CODE HERE
features = ["Undervote", "Overvote", "Bush00", "Gore00", "Buchanan00", "Other00"] # Features for total votes in county calculation

df["overvote_proportion1"] = (df["Overvote"] / df[features].sum(axis="columns")) # proportion of overvotes in a county
df["overvote_proportion2"] = (df["Overvote"] / df["Overvote"].sum()) # proportion of overvotes out of total overvotes


print(f" Overvote/all voters in county:\n{df[["County", "overvote_proportion1"]]}")
print(f"\n Overvote/total_overvotes for each county:\n{df[["County", "overvote_proportion2"]]}")

 Overvote/all voters in county:
                County  overvote_proportion1
0              Alachua              0.001220
1                Baker              0.005556
2                  Bay              0.002370
3             Bradford              0.073865
4              Brevard              0.000622
..                 ...                   ...
63             Volusia              0.000929
64             Wakulla              0.041412
65              Walton              0.003887
66          Washington              0.004303
67  (Federal Absentee)                   NaN

[68 rows x 2 columns]

 Overvote/total_overvotes for each county:
                County  overvote_proportion2
0              Alachua              0.000923
1                Baker              0.000404
2                  Bay              0.001239
3             Bradford              0.006106
4              Brevard              0.001195
..                 ...                   ...
63             Volusia              0.001502
6

Question 3b
In the 2000 election, every county in Florida decided on a different voting machine and ballot design.

Use the grammar of graphics to determine if there is evidence that the voting machine (VotingMachine) or ballot design (Ballot) influenced the overvote ratio defined above.

In [138]:
# YOUR CODE HERE
import plotly.express as px

fig1 = px.bar(df.groupby("VotingMachine")["Overvote"].mean().reset_index(),
       x="VotingMachine", y="Overvote")

fig1.show()

fig2 = px.bar(df.groupby("Ballot")["Overvote"].mean().reset_index(),
       x="Ballot", y="Overvote")

fig2.show()

YOUR INTERPRETATION HERE

Yes, from the graphics above we can clearly see that ballot types and voting machine types influnce the overvote proportion.

Question 4
Al Gore challenged the state-certified count in court, in a case that made it all the way to the Supreme Court (Bush v. Gore). Since miscast votes and overvotes cannot be counted, Gore's challenge focused on undervotes. Undervoting is when a voter selects no candidate. However, this may be because the voter did not fully punch a hole in the ballot, leaving so-called "hanging chads" (see image).
[hanging_chad.jpg](attachment:04f47417-73df-4d13-9e90-641667d38d35.jpg)
Gore demanded recounts in counties with punch-card ballots (i.e., Votomatic or Datavote), arguing that the undervote rate was higher in counties with punch-card ballots. Do you agree?!

In [139]:
# YOUR CODE HERE
import plotly.express as px

fig1 = px.bar(df.groupby("VotingMachine")["Undervote"].mean().reset_index(),
       x="VotingMachine", y="Undervote")

fig1.show()


YOUR INTERPRETATION HERE

We can clearly see that using Votomatic influences undervotes but Datavote does not state this. In this case, yes, I am partly agree that using of hanging chads leads to increase of undervotes in some conditions. Using of Votomatic increases both overvotes and undervotes.